In [1]:
import pandas as pd

In [56]:
cleaned = pd.read_json('cleaned2.json')

In [57]:
file_path = 'Message_1.txt'

with open(file_path, 'r') as file:
    content = file.read()
hm = dict()
with open(file_path, 'r') as file:
    for line in file:
        # Process each line here
        arr = line.split(',') 
        arr[-1] = arr[-1].strip() 
        hm[arr[0]] = tuple(arr[1:])

def get_unique_countries(row):
    return list({e[1] for e in row})

cleaned['Country'] = cleaned['affiliation'].map(get_unique_countries)
cleaned = cleaned.explode('Country')
cleaned['Coordinates'] = cleaned['Country'].map(hm)




In [59]:
df_coord = pd.DataFrame((cleaned['Coordinates']).tolist(), columns=['Latitude', 'Longitude'], index=cleaned.index)

df_coord['classification']=  cleaned['classification']

def extract(row):
    filtered = [e["classification"] for e in row if e["@type"] == "SUBJABBR"][0]
    if not filtered:
        return pd.DataFrame()
    if isinstance(filtered, list):
        return [e["$"] for e in filtered]
    return [filtered]

df_coord['classification'] = df_coord['classification'].map(extract)
df_coord.reset_index(drop=True).to_json('coord.json')